In [2]:
from dotenv import load_dotenv
import os
import openai 
import pandas as pd 

# Carregar variáveis do arquivo .env
load_dotenv()

# Verificar se a chave foi carregada
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# Configurar o cliente OpenAI com a chave carregada
openai.api_key = api_key

In [3]:
client = openai.Client()

In [5]:
file = client.files.create(
    file=open("sales_data.csv","rb"),
    purpose="assistants"
)

In [6]:
print(file.id)

file-6uRC9wHJRm8vY9EAoJ5Jdc


In [11]:
assistant = client.beta.assistants.create(
    name="Analista de dados",
    instructions="Voce é um analsita que analisa dados sobre vendas",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o"
)

In [20]:
# pergunta = "Qual o rating médio das vendas do supermercado"
pergunta = "Gere um gráfico de pizza com o percentual de vendas por linha de produto"

In [21]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [22]:
# Executa a Thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome do usuário premium"
)

In [23]:
# Aguarda a thread rodar
import time 
while run.status in ["queued","in_progress","cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [24]:
# Verificar a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_DhGTQOwBTjaIXhmUPuAEzaM0', assistant_id='asst_DgtnZka7CfxNqwT2zDKL7DwH', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-UB3rKzh9QtSa5MMWFBtX2B', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de pizza que mostra o percentual de vendas por linha de produto. Cada segmento representa a proporção das vendas totais correspondentes a cada linha de produto. Se você precisar de mais alguma ajuda ou de outra visualização, é só avisar!'), type='text')], created_at=1737750392, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_qNRYBdfEsA9wIoUw3MklDqvd', status=None, thread_id='thread_G7KV1hshR0Lg6xXYWHwUksmY'), Message(id='msg_MFbCaNyBTFeAxp7GEY06lf6U', assistant_id='asst_DgtnZka7CfxNqwT2zDKL7DwH', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[

In [32]:
#mensagens.data[0].content[0].text.value

In [26]:
# Analisando os passos do modelo

run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [31]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} salva")
        
        # print(message.content[0].text.value)


====Step message_creation
Vamos começar carregando o arquivo enviado para verificar os dados que ele contém. Isso nos ajudará a identificar as colunas relevantes para gerar o gráfico de pizza com o percentual de vendas por linha de produto.

====Step tool_calls
import pandas as pd

# Carregar o arquivo para inspeção
file_path = '/mnt/data/file-6uRC9wHJRm8vY9EAoJ5Jdc'
data = pd.read_csv(file_path)

# Exibir as primeiras linhas do dataframe para entender sua estrutura
data.head()

====Step message_creation
O dataframe contém várias colunas, e podemos observar que a coluna relevante para nossa análise é a `"Product line"`, que representa as linhas de produtos das vendas, e a coluna `"Total"`, que indica o valor total das vendas. Vamos agora somar os valores das vendas por linha de produto e, em seguida, criar um gráfico de pizza para exibir o percentual de vendas por cada linha de produto.

====Step tool_calls
import matplotlib.pyplot as plt

# Calcular a soma das vendas por linha de pro